In [1]:
from pydantic_settings import BaseSettings, SettingsConfigDict

class Settings(BaseSettings):
    model_config = SettingsConfigDict(
        env_file="../.env", env_file_encoding="utf-8", extra="ignore"
    )
    mineru_model_dir: str
settings = Settings()

# 1. Download mineru weights

In [2]:
import json
import os

import requests
from huggingface_hub import snapshot_download

In [3]:
# download code from https://github.com/opendatalab/MinerU/blob/master/scripts/download_models_hf.py
def download_json(url):
    response = requests.get(url)
    response.raise_for_status() 
    return response.json()


def download_and_modify_json(url, local_filename, modifications):
    if os.path.exists(local_filename):
        data = json.load(open(local_filename))
        config_version = data.get('config_version', '0.0.0')
        if config_version < '1.0.0':
            data = download_json(url)
    else:
        data = download_json(url)

    for key, value in modifications.items():
        data[key] = value

    with open(local_filename, 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=4)

In [4]:
## pdf-extract-kit
mineru_patterns = [
    "models/Layout/LayoutLMv3/*",
    "models/Layout/YOLO/*",
    "models/MFD/YOLO/*",
    "models/MFR/unimernet_small/*",
    "models/TabRec/TableMaster/*",
    "models/TabRec/StructEqTable/*",
]
model_dir = snapshot_download(
    'opendatalab/PDF-Extract-Kit-1.0',
    allow_patterns=mineru_patterns,
    local_dir=os.path.join(settings.mineru_model_dir, 'pdf-extract-kit'),
    local_dir_use_symlinks=False
)
print(f'model_dir is: {model_dir}')

/opt/miniconda3/envs/MinerU/lib/python3.10/site-packages/huggingface_hub/file_download.py:832: UserWarning: `local_dir_use_symlinks` parameter is deprecated and will be ignored. The process to download files to a local folder has been updated and do not rely on symlinks anymore. You only need to pass a destination folder as`local_dir`.
For more details, check out https://huggingface.co/docs/huggingface_hub/main/en/guides/download#download-files-to-local-folder.
  warnings.warn(


Fetching 42 files:   0%|          | 0/42 [00:00<?, ?it/s]

model_dir is: /Users/id4thomas/models/mineru/pdf-extract-kit


In [5]:
layoutreader_pattern = [
    "*.json",
    "*.safetensors",
]
layoutreader_model_dir = snapshot_download(
    'hantian/layoutreader',
    allow_patterns=layoutreader_pattern,
    local_dir=os.path.join(settings.mineru_model_dir, 'layoutreader'),
    local_dir_use_symlinks=False
)

print(f'layoutreader_model_dir is: {layoutreader_model_dir}')

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

layoutreader_model_dir is: /Users/id4thomas/models/mineru/layoutreader


In [6]:
json_url = 'https://github.com/opendatalab/MinerU/raw/master/magic-pdf.template.json'
config_file_name = 'magic-pdf.json'
config_file = os.path.join(settings.mineru_model_dir, config_file_name)

json_mods = {
    'models-dir': os.path.join(settings.mineru_model_dir, 'pdf-extract-kit/models'),
    'layoutreader-model-dir': os.path.join(settings.mineru_model_dir, 'layoutreader'),
}

download_and_modify_json(json_url, config_file, json_mods)
print(f'The configuration file has been configured successfully, the path is: {config_file}')

The configuration file has been configured successfully, the path is: /Users/id4thomas/models/mineru/magic-pdf.json


# 2. mineru test
* https://mineru.readthedocs.io/en/latest/user_guide/quick_start/to_markdown.html

In [7]:
os.environ["MINERU_TOOLS_CONFIG_JSON"] = "models/mineru/magic-pdf.json"

In [8]:
from magic_pdf.data.data_reader_writer import FileBasedDataWriter, FileBasedDataReader
from magic_pdf.data.dataset import PymuDocDataset
from magic_pdf.model.doc_analyze_by_custom_model import doc_analyze
from magic_pdf.config.enums import SupportedPdfParseMethod

import tensorrt_llm failed, if do not use tensorrt, ignore this message
import lmdeploy failed, if do not use lmdeploy, ignore this message


In [16]:
fname = "1706.03762v7"
fname = "ai_parl_2" # 국회도서관 문서 - 이미지 PDF
fname = "ai_parl_3" # 국회도서관 문서 - 텍스트 PDF

pdf_file_name = f"samples/{fname}.pdf"
name_without_suff = pdf_file_name.split(".")[0]

In [17]:
local_image_dir, local_md_dir = f"results/mineru/{fname}/images", f"results/mineru/{fname}/output"
image_dir = str(os.path.basename(local_image_dir))
os.makedirs(local_image_dir, exist_ok=True)

In [18]:
image_writer, md_writer = FileBasedDataWriter(local_image_dir), FileBasedDataWriter(
    local_md_dir
)
image_dir = str(os.path.basename(local_image_dir))

In [19]:
reader1 = FileBasedDataReader("")
pdf_bytes = reader1.read(pdf_file_name) 

In [20]:
ds = PymuDocDataset(pdf_bytes)
print(ds.classify())

2025-01-08 23:52:44.892 | INFO     | magic_pdf.libs.pdf_check:detect_invalid_chars:57 - cid_count: 0, text_len: 9614, cid_chars_radio: 0.0


SupportedPdfParseMethod.TXT


In [21]:
## inference
if ds.classify() == SupportedPdfParseMethod.OCR:
    print("OCR MODE")
    infer_result = ds.apply(doc_analyze, ocr=True)
    pipe_result = infer_result.pipe_ocr_mode(image_writer)
else:
    print("TEXT MODE")
    infer_result = ds.apply(doc_analyze, ocr=False)
    pipe_result = infer_result.pipe_txt_mode(image_writer)

2025-01-08 23:52:45.529 | INFO     | magic_pdf.libs.pdf_check:detect_invalid_chars:57 - cid_count: 0, text_len: 9614, cid_chars_radio: 0.0
2025-01-08 23:52:45.535 | INFO     | magic_pdf.model.pdf_extract_kit:__init__:78 - DocAnalysis init, this may take some times, layout_model: layoutlmv3, apply_formula: True, apply_ocr: False, apply_table: False, table_model: rapid_table, lang: None
2025-01-08 23:52:45.535 | INFO     | magic_pdf.model.pdf_extract_kit:__init__:91 - using device: cpu
2025-01-08 23:52:45.535 | INFO     | magic_pdf.model.pdf_extract_kit:__init__:95 - using models_dir: /Users/id4thomas/models/mineru/pdf-extract-kit/models
2025-01-08 23:52:45.535 | INFO     | magic_pdf.model.pdf_extract_kit:__init__:170 - DocAnalysis init done!
2025-01-08 23:52:45.536 | INFO     | magic_pdf.model.doc_analyze_by_custom_model:custom_model_init:181 - model init cost: 0.0040740966796875


TEXT MODE


2025-01-08 23:52:53.866 | INFO     | magic_pdf.model.pdf_extract_kit:__call__:202 - layout detection time: 8.3
2025-01-08 23:52:56.213 | INFO     | magic_pdf.model.pdf_extract_kit:__call__:210 - mfd time: 2.34
2025-01-08 23:53:00.025 | INFO     | magic_pdf.model.pdf_extract_kit:__call__:217 - formula nums: 5, mfr time: 3.81
2025-01-08 23:53:01.117 | INFO     | magic_pdf.model.pdf_extract_kit:__call__:251 - det time: 1.09
2025-01-08 23:53:01.118 | INFO     | magic_pdf.model.doc_analyze_by_custom_model:doc_analyze:224 - -----page_id : 0, page total time: 15.55-----
2025-01-08 23:53:07.693 | INFO     | magic_pdf.model.pdf_extract_kit:__call__:202 - layout detection time: 6.55
2025-01-08 23:53:09.983 | INFO     | magic_pdf.model.pdf_extract_kit:__call__:210 - mfd time: 2.29
2025-01-08 23:53:14.340 | INFO     | magic_pdf.model.pdf_extract_kit:__call__:217 - formula nums: 8, mfr time: 4.36
2025-01-08 23:53:15.216 | INFO     | magic_pdf.model.pdf_extract_kit:__call__:251 - det time: 0.88
2025

In [22]:
### draw model result on each page
infer_result.draw_model(os.path.join(local_md_dir, f"{fname}_model.pdf"))

### draw layout result on each page
pipe_result.draw_layout(os.path.join(local_md_dir, f"{fname}_layout.pdf"))

### draw spans result on each page
pipe_result.draw_span(os.path.join(local_md_dir, f"{fname}_spans.pdf"))

### dump markdown
pipe_result.dump_md(md_writer, f"{fname}.md", image_dir)

### dump content list
pipe_result.dump_content_list(md_writer, f"{fname}_content_list.json", image_dir)